In [1]:
import keras
from keras import layers
import keras_tuner as kt
from emotion_model_pipeline import load_data
import tensorflow as tf

# Load the data
data = load_data(
    X_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/X.npy',  # Path to the features file
    y_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/y.npy',  # Path to the labels file
)

# Print the data shapes
print(f"Training data shape: {data['X_train'].shape}")
print(f"Testing data shape: {data['X_test'].shape}")

# Normalize the input data
#X_train = data['X_train'] / 255.0  # Normalize pixel values to range [0, 1]
#X_val = data['X_test'] / 255.0
#y_train = data['y_train']
#y_val = data['y_test']


Training data shape: (20973, 48, 48, 1)
Testing data shape: (5244, 48, 48, 1)


def build_model(hp):
    model = keras.Sequential()

    # First Convolutional Layer
    model.add(layers.InputLayer(input_shape=(48, 48, 1)))
    model.add(layers.Conv2D(
        filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Layer
    model.add(layers.Conv2D(
        filters=hp.Int('filters_2', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten and Dense Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),
        activation='relu'
    ))

    # Output Layer (4 classes for 4 emotions)
    model.add(layers.Dense(4, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='sparse_categorical_crossentropy',  # For integer labels
        metrics=['accuracy']
    )

    return model


tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # Maximize validation accuracy
    max_trials=10,  # Number of trials
    executions_per_trial=3,  # Number of executions per trial
    directory='kt_dir',  # Directory to store the results
    project_name='emotion_classification'
)


tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


import mlflow
import mlflow.keras

def log_trial_in_mlflow(trial):
    # Log the hyperparameters and metrics of the trial in MLflow
    with mlflow.start_run():
        mlflow.log_param('filters_1', trial.params['filters_1'])
        mlflow.log_param('filters_2', trial.params['filters_2'])
        mlflow.log_param('kernel_size_1', trial.params['kernel_size_1'])
        mlflow.log_param('kernel_size_2', trial.params['kernel_size_2'])
        mlflow.log_param('dense_units', trial.params['dense_units'])
        mlflow.log_param('learning_rate', trial.params['learning_rate'])
        mlflow.log_metric('val_accuracy', trial.results['val_accuracy'])
        
        # Log the best model
        mlflow.keras.log_model(model, "model")

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hyperparameters.values)

# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hyperparameters)

# Train the final model
best_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


# Evaluate the model on the test set
test_loss, test_acc = best_model.evaluate(data['X_test'], data['y_test'])
print(f"Test accuracy: {test_acc}")

# Save the model
best_model.save('emotion_classification_model.h5')


In [21]:
import keras
from keras import layers
import keras_tuner as kt
import mlflow
import mlflow.keras
from emotion_model_pipeline import load_data
import tensorflow as tf

In [23]:
#!pip install mlflow

In [25]:
# Define the model-building function with hyperparameter tuning
def build_model(hp):
    model = keras.Sequential()
    
    # Choose number of convolutional layers
    model.add(layers.InputLayer(input_shape=(48, 48, 1)))
    
    for i in range(hp.Int('conv_layers', 1, 3)):  # Randomly choose number of conv layers (1 to 3)
        model.add(layers.Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32),  # Random filter size
            kernel_size=(3, 3),
            activation='relu'
        ))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Flatten())
    
    # Add a fully connected layer with number of units
    model.add(layers.Dense(
        hp.Int('units', min_value=64, max_value=512, step=64),  # Random number of units
        activation='relu'
    ))

    # Output layer with 4 classes (emotion classification)
    model.add(layers.Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [27]:
import mlflow

# Define your experiment name
experiment_name = "emotion_recognition_experiment"

# Set the experiment
mlflow.set_experiment(experiment_name)

# Verify the experiment was created (you can print the current experiment to check)
experiment = mlflow.get_experiment_by_name(experiment_name)
print(experiment)


<Experiment: artifact_location='file:///C:/Users/Surface/NU/Vibify/pipelines/dags/src/mlruns/854121127519128219', creation_time=1731689171128, experiment_id='854121127519128219', last_update_time=1731689171128, lifecycle_stage='active', name='emotion_recognition_experiment', tags={}>


In [15]:
print(experiment)

<Experiment: artifact_location='file:///C:/Users/Surface/NU/Vibify/pipelines/dags/src/mlruns/854121127519128219', creation_time=1731689171128, experiment_id='854121127519128219', last_update_time=1731689171128, lifecycle_stage='active', name='emotion_recognition_experiment', tags={}>


In [19]:
# Function to log results to MLFlow
def log_trial_to_mlflow(trial):
    with mlflow.start_run():
        # Log hyperparameters
        for hp_name, value in trial.hyperparameters.values.items():
            mlflow.log_param(hp_name, value)

        # Log the metrics (e.g., validation accuracy)
        mlflow.log_metric('val_accuracy', trial.score)

In [25]:
# Load the data
data = load_data(
    X_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/X.npy',  # Path to the features file
    y_path='C:/Users/Surface/NU/Vibify/dags/data/preprocessed/y.npy',  # Path to the labels file
)

# Print the data shapes
print(f"Training data shape: {data['X_train'].shape}")
print(f"Testing data shape: {data['X_test'].shape}")

Training data shape: (20973, 48, 48, 1)
Testing data shape: (5244, 48, 48, 1)


In [27]:
# Initialize the RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # Objective to optimize
    max_trials=1,  # Number of trials to run
    executions_per_trial=1,  # Number of executions per trial
    directory='tuner_dir',  # Directory to store results
    project_name='emotion_model_random_search'  # Project name for the tuning
)

In [31]:
tuner.search(
    data['X_train'], data['y_train'], 
    epochs=1, 
    validation_data=(data['X_test'], data['y_test'])
    )

Trial 2 Complete [00h 00m 59s]
val_accuracy: 0.4742562919855118

Best val_accuracy So Far: 0.563787192106247
Total elapsed time: 00h 03m 17s


In [33]:
for trial in tuner.oracle.get_best_trials():
    log_trial_to_mlflow(trial)

# Optionally, save the best model
best_model = tuner.get_best_models()[0]
mlflow.keras.log_model(best_model, 'best_model')

C:\Users\Surface\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
2024/11/15 11:25:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/15 11:25:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
